![](img/575_banner.png)

# Lecture 7: More RNNs, LSTMs

UBC Master of Data Science program, 2021-22

Instructor: Varada Kolhatkar

## Lecture plan, imports, LO

### Lecture plan 

- Lab 4 announcement 
- RNN Toy example
- Stacked and bidirectional RNNs
- LSTM motivation 
- iClicker questions
- Break
- LSTMs
- LSTM toy example
- Final comments and summary

<br><br>

## Imports

In [1]:
import sys
from collections import defaultdict

import torch
import torch.nn as nn
from torch.nn import RNN

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.set_option("display.max_colwidth", 0)

<br><br>

### Learning outcomes

From this lecture you will be able to 

- Broadly explain character-level text generation with RNNs;
- Specify the shapes of weight matrices in RNNs;
- Carry out forward pass with RNNs in `PyTorch`;
- Explain stacked RNNs and bidirectional RNNs and the difference between the two;
- Broadly explain the problem of vanishing gradients; 
- Broadly explain the idea of LSTMs at a high level;
- Carry out forward pass with LSTMs in `PyTorch`;
- Explain the shapes of input, output, hidden state and cell state in LSTMs;
- Broadly explain numericalization, batching, and padding in text preprocessings;
- Explain the motivation to use `torchtext`.

<br><br>

### Attributions

This material is heavily based on [Jurafsky and Martin, Chapter 9]((https://web.stanford.edu/~jurafsky/slp3/9.pdf)).

<br><br><br><br>

## Recap and motivation 

### RNN Toy example with PyTorch

### From the last lecture

- We know basics of RNNs. 
- Now we'll look at a toy example for character-level text generation using RNNs. 
- Recall that given a sequence of characters, character-level text generation is the task of modeling probability distribution of the next character in the sequence. 

### A toy "hello" RNN 
- Suppose we want to train a character-level RNN on sequence "hello". 
- The vocabulary is 4 and we want our model to learn the following: 
    - "e" should be likely given "h" 
    - "l" should be likely given "he" 
    - "l" should be likely given "hel" 
    - "o" should be likely given "hell"     

![](img/RNN_char_generation_train.png)

<!-- <center> -->
<!-- <img src="img/RNN_char_generation_train.png" height="500" width="500">  -->
<!-- <center>     -->

[Source](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Shapes of input, hidden, and output weight matrices
- Shape of $W_{xh}$ ($U$) is going to be: $4 \times 3$
- Shape of $W_{hh}$ ($W$) is going to be: $3 \times 3$
- Shape of $W_{hy}$ ($V$) is going to be: $3 \times 4$
$$
s_t = g(Ws_{t-1} + Ux_t + b_1)\\
\hat{y}_t = \text{softmax}(Vs_t + b_2)
$$ 

![](img/RNN_char_generation_train.png)

<!-- <center> -->
<!-- <img src="img/RNN_char_generation_train.png" height="600" width="600">  -->
<!-- <center>   -->

[Source](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Let's build a simple RNN for this using `PyTorch`. 

In [2]:
torch.manual_seed(123)

- Let's define a mapping between indices and characters. 

In [3]:
idx2char = ["h", "e", "l", "o"]

We need some representation for the input. Let's use one-hot representation. 

In [4]:
one_hot_lookup = [
    [1, 0, 0, 0],  # h
    [0, 1, 0, 0],  # e
    [0, 0, 1, 0],  # l
    [0, 0, 0, 1],  # o
]

Next let's create one-hot representation of `X`. 

In [5]:
X = [0, 1, 2, 2]  # indices for the input "hell"
X_one_hot = [one_hot_lookup[x] for x in X]
inputs = torch.Tensor(X_one_hot)
inputs

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]])

In [6]:
y = [1, 2, 2, 3]
labels = torch.LongTensor(y)
labels

tensor([1, 2, 2, 3])

### Defining some variables 

In [7]:
num_classes = 4  # size of vocab
EPOCHS = 10  # number of epochs
input_size = 4  # size of vocab or one-hot size
hidden_size = 3  # output from the RNN.
batch_size = 1  # we are not batching in this toy example.
sequence_length = 1  # we are processing characters one by one in this toy example
num_layers = 1  # one-layer rnn

![](img/RNN_char_generation_train.png)

<!-- <center> -->
<!-- <img src="img/RNN_char_generation_train.png" height="500" width="500">  -->
<!-- <center>     -->

[Source](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [8]:
class ToyRNN(nn.Module):
    def __init__(self, debug=False):
        super(ToyRNN, self).__init__()

        # PyTorch core RNN module
        self.rnn = nn.RNN(
            input_size=input_size, hidden_size=hidden_size, batch_first=True
        )

        # Fully connected layer for the output
        self.fc = nn.Linear(hidden_size, num_classes)

        # Debugging flag
        self.debug = debug

    def forward(self, hidden, x):
        x = x.view(batch_size, sequence_length, input_size)  # reshape the input
        if self.debug:
            print("\n\n")
            print("Input shape = ", x.size())

        out, hidden = self.rnn(x, hidden)
        if self.debug:
            print("out shape = ", out.size())
            print("Hidden shape = ", hidden.size())

        out = out.reshape(out.shape[0], -1)  # reshape to pass before the output layer
        if self.debug:
            print("out shape after reshaing = ", out.size())

        out = self.fc(out)
        if self.debug:
            print("out shape after passing through fc = ", out.size())

        return hidden, out

    def init_hidden(self):
        return torch.zeros(num_layers, batch_size, hidden_size)

### Instantiate the model

In [9]:
model = ToyRNN()
print(model)

# Set loss and optimizer function
# Loss increases as the predicted probability diverges from the actual label.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

ToyRNN(
  (rnn): RNN(4, 3, batch_first=True)
  (fc): Linear(in_features=3, out_features=4, bias=True)
)


### Train the model

In [10]:
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()

    pred = ""
    for inp, label in zip(inputs, labels):
        hidden, output = model(hidden, inp)
        val, idx = output.max(1)
        pred += idx2char[idx.data[0]]
        loss += criterion(output, torch.LongTensor([label]))
    print("Epoch: %d, loss: %1.3f, preidcted: %s" % (epoch + 1, loss, pred))

    loss.backward()
    optimizer.step()

Epoch: 1, loss: 6.082, preidcted: oeoe
Epoch: 2, loss: 5.008, preidcted: olol
Epoch: 3, loss: 4.393, preidcted: llll
Epoch: 4, loss: 4.155, preidcted: llll
Epoch: 5, loss: 3.991, preidcted: llll
Epoch: 6, loss: 3.697, preidcted: llll
Epoch: 7, loss: 3.280, preidcted: llll
Epoch: 8, loss: 2.864, preidcted: ello
Epoch: 9, loss: 2.459, preidcted: ello
Epoch: 10, loss: 2.020, preidcted: ello


![](img/RNN_char_generation_train.png)

<!-- <center> -->
<!-- <img src="img/RNN_char_generation_train.png" height="600" width="600">  -->
<!-- <center>     -->

[Source](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

- We have our toy RNN for text generation! 
- Usually we would do it on large text corpora (e.g., the whole Wikipedia or The New York Times articles from the last 20 years). 

<br><br><br><br>

## Stacked and Bidirectional RNN architectures

- We have seen a simple RNN with one hidden layer. 
- But RNNs are quite flexible. 
- Two common ways to create complex networks by combining RNNs are:
    - Stacked RNNs
    - Bidirectional RNNs

### Stacked RNNs 

- In the examples thus far, the input of RNNs was a sequence of word or character embeddings. We were passing the output of the RNN layer to the output layer and the outputs have been vectors useful for predicting next words, tags, or sequence labels.  

![](img/RNN_seq_labeling.png)


[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- But nothing prevents us from using **the sequence of outputs from one RNN as an input sequence to another one**.
- These are called **stacked RNNs** which consist of multiple networks where the output of one layer serves as the input to a subsequent layer. 

![](img/RNN_stacked.png)

<!-- ![](img/RNN_stacked.png) -->

<!-- <img src="img/RNN_stacked.png" height="800" width="800">  -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- Stacked RNNs generally outperform single-layer networks. 
- The network learns a different level of abstraction at each layer. 
- You can optimize your network for number of layers for your specific application and dataset.  
- But remember that more layers means higher training cost. 

### Bidirectional RNNs 

- The RNN uses information from the prior context to make predictions at time $t$. 
- But in many applications (e.g., POS tagging) we do have access to the entire input sequence and knowing the context on the right of time $t$ can be useful. 
- For example, suppose you are doing POS tagging and you are at the token **Teddy** in the sequence. It will be useful to know the right context in order to make the decision on whether it should be tagged as a _noun_ or a _proper noun_.  

> He said , " Teddy Roosevelt was a great president ! "<br>

> He said , " Teddy bears are on sale ! "


- How can we use the words on the right of time step $t$ as context?  
- In the left-to-right RNN, the hidden state at time $t$ represents everything the network knows about the sequence up to that point. 
- Suppose $h_t^f$ denotes a hidden state at time $t$ representing everything the network has gleaned from the sequence so far. 
$$h_t^f = RNN_{forward}(x_1, x_2, \dots, x_t) $$
- We can also train the network in the reverse direction, from right to left, to take advantage of the right context. 
- With this approach the hidden state at time $t$, $h_t^b$ represents all the information we have learned about the sequence from time $t$ to the end of the sequence. 
$$h_t^b = RNN_{backward}(x_t, x_{t+1}, \dots, x_n) $$
- (Somewhat similar to the $\alpha$ and $\beta$ values in the forward and backward algorithms in HMMs.)

A **bidirectional RNN** combines two independent RNNs:
- One where the input is processed from the start to the end
- The other from the end to the start. 
- Each RNN will result in some representation of the input. 
- We then combine the two representations computed by two independent RNNs into a single vector which captures both the left and right contexts of an input at each point in time. 
- We can combine vectors by
    - Concatenating them, as shown in the picture below or
    - Element-wise addition 
    - Element-wise multiplication

![](img/bidirectional_seq_labeling.png)
<!-- <img src="img/bidirectional_seq_labeling.png" height="800" width="800">  -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- You can also use bidirectional RNNs for sequence classification. 
- Recall that in sequence classification we pass the final hidden state of the RNN as input to a subsequent feedforward classifier. 
- The problem with this approach is that the final hidden state reflects more information about the end of the sequence than its beginning. 
- Bidirectional RNNs provide a simple solution to this problem. We can create a final hidden state by combining hidden states of forward and backward passes so that the hidden state reflects information about both the beginning and end of the sequence. 

![](img/bidirectional_classification.png)
<!-- <img src="img/bidirectional_classification.png" height="800" width="800">  -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

<br><br><br><br>

## LSTM motivation 

### Problems with RNNs 

- In practice, you'll hardly see people using vanilla RNNs because they are quite hard to train for tasks that require access to distant information. 
- Despite having access to the entire previous sequence, the information encoded in hidden states of RNNs is fairly local.

#### Problem 1

Consider the examples below in the context of language modeling. 

> The students in the exam where the fire alarm is ringing **are** really stressed. 

> The flies munching on the banana that is lying under the tree which is in full bloom **are** really happy. 

- Assigning high probability to **_is_** following *alarm* is straightforward since it provides a local context for singular agreement. 
- However, assigning a high probability to **_are_** following _ringing_ is quite difficult because not only the plural _students_ is distant, but also the intervening context involves singular constituents. 
- Ideally, we want the network to retain the distant information about the plural **_students_** until it's needed while still processing the intermediate parts of the sequence correctly. 

- The hidden layer and the weights that determine the values in the hidden layer are asked to perform two tasks simultaneously:
    - Providing information useful for current decision
    - Updating and carrying forward information required for future decisions

#### Problem 2: Vanishing gradients

- Another difficulty with training RNNs arises from the need to backpropagate the error signal back through time. 
- Recall that we learn RNNs with 
    - Forward pass 
    - Backward pass (backprop through time)
    
- Computing new states and output in RNNs

$$
s_t = g(Ws_{t-1} + Ux_t + b_1)\\
\hat{y}_t = \text{softmax}(Vs_t + b_2)
$$ 

![](img/RNN_dynamic_model.png)

<!-- <center> -->
<!-- <img src="img/RNN_dynamic_model.png" height="800" width="800">  -->
<!-- </center>     -->

Recall: Backpropagation through time

- When we do backprop with feedforward neural networks
    - Take the gradient (derivative) of the loss with respect to the parameters. 
    - Change parameters to minimize the loss. 

- In RNNs we use a generalized version of backprop called Backpropogation Through Time (BPTT)
    - Calculating gradient at each output depends upon the current time step as well as the previous time steps. 

![](img/RNN_loss.png)

<!-- <center>     -->
<!-- <img src="img/RNN_loss.png" height="800" width="800">  -->
<!-- </center> -->
    
[Credit](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)

- So in the backward pass of RNNs, we have to multiply many derivatives together, which very often results in
    - vanishing gradients (gradients becoming very small and eventually driven to zero) in case of long sequences
- If we have a vanishing gradient, we might not be able to update our weights reliably. 
- Only able to capture short-term dependencies, which kind of defeats the whole purpose of using RNNs.     

- To address these issues more complex network architectures have been designed with the goal of maintaining relevant context over time by enabling the network to learn to forget the information that is no longer needed and to remember information required for decisions still to come. 
- One of the most commonly used such models are 
    - The Long short-term memory network (LSTM)
    - Gated Recurrent Units (GRU)
- Let's look at some example applications of LSTMs.     

### LSTM for image captioning 

![](img/RNN_LSTM_image_captioning.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM_image_captioning.png" height="2000" width="2000">  -->
<!-- </center>     -->


(Credit: [LSTMs for image captioning](https://arxiv.org/pdf/1411.4555.pdf))

[Neural Storyteller](https://github.com/ryankiros/neural-storyteller)

![](img/RNN_example.jpg)
<!-- <img src="img/RNN_example.jpg" width="500" height="500"> -->


<blockquote>        
<p style="font-size:30px">We were barely able to catch the breeze at the beach , and it felt as if someone stepped out of my mind . She was in love with him for the first time in months , so she had no intention of escaping . The sun had risen from the ocean , making her feel more alive than normal . She 's beautiful , but the truth is that I do n't know what to do ...</p>
</blockquote>    

### LSTMs for video captioning 

![](img/RNN_video_captioning2.png)

<!-- <center> -->
<!-- <img src="img/RNN_video_captioning2.png" height="1000" width="1000">  -->
<!-- </center> -->

(Credit: [LSTMs for video captioning](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Venugopalan_Sequence_to_Sequence_ICCV_2015_paper.pdf))

### LSTMs for executing Python programs 

- In 2014, Google researchers built an LSTM that learns to execute simple
Python programs!

![](img/RNN_learning_to_execute.png)

<!-- <img src="img/RNN_learning_to_execute.png" width="1500" height="1500"> -->

(Credit: [Learning to execute](https://arxiv.org/pdf/1410.4615.pdf))

<br><br><br><br>

## ❓❓ Questions for you

iClicker cloud join link: https://join.iclicker.com/4QVT4

### Exercise 7.1: Select all of the following statements which are **True** (iClicker)

Suppose you are training a Vanilla RNN with one hidden layer. Your input representation is of size 200, output layer is of size 200, and you have 100 hidden units in your hidden layer. 

- (A) The shape of matrix $W$ or $W_{hh}$ between hidden layers in consecutive time steps is going to be $200 \times 200$. 
- (B) The output of the hidden layer is going to be a $100$ dimensional vector.  
- (C) In bidirectional RNNs, if we want to combine the outputs of two RNNs with pointwise addition, the hidden sizes of the two RNNs have to be the same.  
- (D) Word2vec skipgram model is likely to suffer from the problem of vanishing gradients. 
- (E) In the forward pass, in each time step in RNNs, you calculate the output of the hidden layer by multiplying the input $x$ by the weight matrix $U$ or $W_{xh}$ and applying non-linearity. 

<br><br><br><br>

```{admonition} Exercise 7.1: V's Solutions!
:class: tip, dropdown
- (A) False. It's going to be $100 times 100$ 
- (B) True
- (C) True
- (D) False
- (E) False
```

### Break

![](img/eva-coffee.png)

<br><br><br><br>

## (Semi-optional) Long short-term memory networks (LSTMs) 

- Recommendation: Go through these notes once to get a general idea of LSTMs. But try not to get bogged down in all the details. 
- I want you to get the general intuition of these models so that you have a general idea of their purpose and how they work. 

### Long Short Term Memory networks (LSTMs)

- [Invented in 1997](https://www.bioinf.jku.at/publications/older/2604.pdf) by Hochreiter and Schmidhuber. 
- Designed so that model can remember things for a long time (hundreds of time steps)! 

### Simple (Vanilla) RNN 

- In a simple RNN, we have a series of these repeating modules.
- How does the information flow in one cell of RNN? 

![](img/RNN_alternative_representation.png)

<!-- <center> -->
<!-- <img src="img/RNN_alternative_representation.png" height="1000" width="1000">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Long Short Term Memory networks (LSTMs)

- In an LSTM, the repeating module is more complicated. 
- LSTMs add an explicit context layer to the architecture for context management. 
- It's connected to the hiddent layer via specialized neural units which selectively control the flow of information using gates. 

![](img/RNN_alternative_representation.png)
![](img/LSTM0.png)
<!-- <center> -->
<!-- <img src="img/RNN_alternative_representation.png" height="600" width="600">  -->

<!-- <img src="img/LSTM0.png" height="700" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### LSTMs: Controlling the flow of information using gates

- LSTMs divide context management into two sub problems: removing information no longer needed from the context and adding information likely to be needed for later decision making. 
- The information is added and removed through a structure called gates. 
- The gates share a common design pattern: each consists of a feedforward layer followed by a sigmoid activation function, followed by a pointwise multiplication with the layer being gated. 
    - The sigmoid layer pushes its output to either 0 or 1, deciding how much of each component should be let through.
    - Combining this with pointwise multiplication has the effect of a binary mask; the values that align with 1 in the mask are passed through unchanged while the values corresponding to lower values are erased.

![](img/RNN_LSTM2.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM2.png" height="800" width="800">  -->
<!-- </center>     -->
   
(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### LSTMs: The core idea 

- The core idea in LSTMs is using a context or cell state (memory cell)
- Information can flow along the memory unchanged. 
- Information can be removed or written to the cells regulated by gates. 

![](img/RNN_LSTM0.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM0.png" height="800" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### How does LSTM work?
- Four operations: forget, store (input), update, output

![](img/RNN_LSTM0.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM0.png" height="800" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Forget operation

- The purpose of this gate is to delete information from the context that is no longer needed. 
- A sigmoid layer, **forget gate**, decides which values of the memory cell to reset. 
- Decides what part of the history is worth forgetting.
- $f_t = \sigma(W_fh_{t-1} + U_fx_t + b_f)$

![](img/RNN_LSTM3.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM3.png" height="800" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Store operation 

- Decides what part of the new information is worth storing. 
- Two parts: 
    - A sigmoid layer, **input gate.**
    - $i_t = \sigma(W_ih_{t-1} + U_ix_t + b_i)$

![](img/RNN_LSTM4.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM4.png" height="800" width="800">  -->
<!-- </center>     -->


(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Vector of new candidate values

- A $tanh$ layer creates a vector of new candidate values $\tilde{c}_t$ to write to the memory cell. 
- $\tilde{c}_t = tanh(W_{c}h_{t-1} + U_{c}x_t + b_c)$ 

![](img/RNN_LSTM5.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM5.png" height="800" width="800">  -->
<!-- </center>     -->


(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Update operation (memory cell update)

- The previous steps decided which values of the memory cell to reset and overwrite. 
- Now the LSTM applies the decisions to the memory cells.
- $c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$ 

![](img/RNN_LSTM6.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM6.png" height="800" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

### Output operation

- The sigmoid layer, **output gate**, decides which values should be sent to the network in the next time step. 
- $o_t = \sigma(W_{o}h_{t-1} + U_ox_t + b_o)$

![](img/RNN_LSTM7.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM7.png" height="800" width="800">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

#### Output update

- The memory cell goes through $tanh$ and is multiplied by the output gate
- $h_t = o_t \odot \tanh(c_t)$

![](img/RNN_LSTM8.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM8.png" height="800" width="800">  -->
<!-- </center>     -->


(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

- So an LSTM unit at one time step takes as input 
    - the context layer
    - the hidden layer from previous time step
    - the current input vector 
- As output it generates
    - Updated context vector
    - Updated hidden vector
- The nice thing is that the complexity in LSTMs is encapsulated within a the basic processing unit allowing us to maintain modularity and experiment with different architectures quite easily. 
- This modularity is the key to the power and widespread applicability of LSTMs!     
- Similar to RNNs it's possible to stack LSTM layers or use it in a bidirectional setting. 
- There are other variants such as Gated Recurrent Units (GRU) which follow similar idea but are less complicated. 

## LSTMs: Confusing diagrams!!!

- LSTMs are not very intuitive.  
- Complicated combination of state in the past, observation at the moment and different ways to either forget the observation or keep it around. 
- Famous for confusing illustrative diagrams. 

![](img/RNN_confusing_LSTMs.png)

<!-- <center> -->
<!-- <img src="img/RNN_confusing_LSTMs.png" height="1000" width="1000">  -->
<!-- </center>     -->



<br><br><br><br>

## Let's build LSTMs!!!

### Creating some data 

In [11]:
# sequence_len, batch_size, input_size
X = torch.randn(3, 4, 10)
X

tensor([[[-2.0929e+00, -8.1991e-01, -4.2104e-01, -9.6200e-01,  1.2825e+00,
           8.7684e-01,  1.6221e+00, -1.4779e+00, -1.7018e+00, -7.4980e-01],
         [-1.1285e+00,  4.1351e-01,  2.8917e-01,  2.2473e+00, -8.0364e-01,
          -2.8084e-01, -4.2036e-01,  1.3111e+00, -2.1993e-01,  2.1895e-01],
         [ 2.0451e-01,  5.1463e-01,  9.9376e-01, -2.5873e-01,  1.5118e-01,
           1.0364e-01, -2.1996e+00, -8.8490e-02, -5.6120e-01,  6.7155e-01],
         [ 6.9330e-01, -9.4872e-01, -1.1440e+00,  2.4362e-01, -5.6734e-02,
           3.7841e-01,  1.6863e+00,  2.5529e-01, -5.4963e-01,  1.0042e+00]],

        [[ 3.5068e-01,  1.5434e+00,  1.4058e-01,  1.0617e+00, -9.9292e-01,
          -1.6025e+00, -1.0764e+00,  9.0315e-01, -1.6461e+00,  1.0720e+00],
         [ 1.5026e+00, -8.1899e-01,  2.6860e-01, -2.2150e+00, -1.3193e+00,
          -2.0915e+00,  1.2767e+00, -9.9480e-01,  1.2176e+00, -2.2817e-01],
         [ 1.3382e+00,  1.9929e+00,  1.3708e+00, -5.0087e-01, -2.3244e+00,
           1.2311

### Defining the LSTM module 
An LSTM is defined by `torch.nn.LSTM` module with the following parameters. 
- `input_size`: the number of features in x (e.g., size of the word embedding)
- `hidden_size`: the number of nodes in the hidden layer
- `num_layers`: number of recurrent layers

In [12]:
lstm_rnn = nn.LSTM(input_size=10, hidden_size=5, num_layers=2)

### Initializing the cell state and hidden state at time step 0

- first dimension: number of LSTM layers
- second dimension: batch_size
- third dimension: hidden_size / number of nodes in a hidden layer

In [13]:
c0 = torch.randn(2, 4, 5)
h0 = torch.randn(2, 4, 5)

### Forward propagation of LSTM 

Unlike RNN, LSTM module takes three inputs 
- the initial hidden state for each element in the batch (t=0)
- initial cell state for each element in the batch     
- the input features (e.g., embeddings)

In [14]:
# h0 and c0 is optional input, defaults to tensor of 0's when not provided
output, (hn, cn) = lstm_rnn(X, (h0, c0))

In [15]:
# output = seq_len, batch_size, hidden_size (output features from last layer of LSTM)
print("output size: ", output.size())

output size:  torch.Size([3, 4, 5])


### This was not that that complicated but when we want to use it on text data it's a quite involved process.

### Text classification using `PyTorch` and `torchtext` 

- A lot of work in building LSTMs for text data goes into data preprocessing and getting the text into the suitable format. 
- There are a number of steps involved in data preprocessing. 
    - [ ] Data splitting (train, valid, test)
    - [ ] Loading the data files
    - [ ] Tokenization
    - [ ] Creating a vocabulary: Creating a list of unique words 
    - [ ] Numericalization: Converting text to a set of integers. 
    - [ ] Word vectors
    - [ ] Embedding lookup 
    - [ ] Batching   

### Text preprocessing

![](img/lstm-preprocess.png)

<!-- <img src="img/lstm-preprocess.png" height="800" width="800">  -->

### `batch_first=True`

![](img/batch_first_true.png)
<!-- <img src="img/batch_first_true.png" height="800" width="800">  -->

### `batch_first=False` (default)

![](img/batch_first_false.png)
<!-- <img src="img/batch_first_false.png" height="800" width="800">  -->

### [`torchtext`](https://pytorch.org/text/stable/index.html)
- There is a library called `torchtext` to help us with these tasks. 
- To use it You will first need to [install `torchtext`](https://pypi.org/project/torchtext/) in the environment.

```
pip install torchtext
```

- I've included an adapted version of [`torchtext` tutorial](AppendixB-torchtext-tutorial.ipynb) from MDS-CL as an appendix. 
- You can also included a demo of [text classification using LSTMs](AppendixC-LSTM-spam-classification.ipynb) as an appendix.   

### RNNs vs. LSTMs 

- RNNs suffer from the problem of vanishing gradients. 
    - Vanishing gradients lead to difficulty in training.  
- We saw that LSTMs mitigate this problem by introducing a cell state and managing the context better. 
- They are better suited for sequence modeling and capturing long-distance dependencies. 

![](img/RNN_alternative_representation.png)
![](img/LSTM0.png)

<!-- <img src="img/RNN_alternative_representation.png" height="400" width="400">  -->

<!-- <img src="img/LSTM0.png" height="400" width="400">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

There are tons of applications of LSTMs. 

### Wiki page on LSTM applications 

![](img/lstm_applications_wiki.png)

<!-- <img src="img/lstm_applications_wiki.png" width="1500" height="1500"> -->

(Credit: [Learning to execute](https://arxiv.org/pdf/1410.4615.pdf))

### LSTM for image captioning 

![](img/RNN_LSTM_image_captioning.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM_image_captioning.png" height="2000" width="2000">  -->
<!-- </center>     -->


(Credit: [LSTMs for image captioning](https://arxiv.org/pdf/1411.4555.pdf))

<br><br>

### music2dance

- [Dance with Melody: An LSTM-autoencoder Approach to Music-oriented Dance Synthesis](https://hcsi.cs.tsinghua.edu.cn/Paper/Paper18/MM18-TANGTAORAN.pdf)
- [Follow the Music: Dance Motion Synthesis Corresponding to Arbitrary Music](https://www.cs.ubc.ca/~rozentil/data/music2dance.pdf) (UBC researchers)

![](img/music2dance.png)

<!-- <center> -->
<!-- <img src="img/music2dance.png" height="1500" width="1500">  -->
<!-- </center>     -->


<br><br><br><br>

## ❓❓ Questions for you

iClicker cloud join link: https://join.iclicker.com/4QVT4

### Exercise 8.1: Select all of the following statements which are **True** (iClicker)

- (A) Input to an RNN or LSTM is a 3-D tensor with three components: sequence length, batch size, and input size.
- (B) In the forward pass of LSTMs, in addition to the previous hidden state we also pass the previous cell state.
- (C) In LSTMs, creating batches with sequences of similar lengths might minimize padding.
- (D) HMMs can handle variable length sequences but LSTMs cannot.   
- (E) It's almost always a good idea to use LSTMs to model text data over bag-of-words models, as they encode sequential information and are likely to beat bag-of-words models. 

<br><br><br><br>

```{admonition} Exercise 8.1: V's Solutions!
:class: tip, dropdown
- (A) True
- (B) True
- (C) True
- (D) False
- (E) False
```

### Exercise 8.2: Questions for discussion

1. Suppose you are using pre-trained embeddings to train a sentiment analysis model with LSTMs on a small corpus shown below. Now given a test sentence "amazing", would you expect the model to classify the sentence correctly? Why or why not? State your assumptions.  

| text        | sentiment           |
| ------------- |:-------------:|
| what a wonderful movie ! | positive|
| boring ... fell asleep a couple of times | negative |
| astonishing ! ! | positive |

2. In text generation using LSTMs or RNNs, why it might be a good idea to sample from the distribution rather than picking the next word with highest probability? 

```{admonition} Exercise 8.2: V's Solutions!
:class: tip, dropdown
1.  Most likely yes. Although the word **amazing** does not occur in the training corpus, we would be incorporating the information in the model, via pre-trained word embeddings, that the word amazing is similar to wonderful and astonishing which are associated with positive sentiment.
2. To add novelty.
```

<br><br><br><br>

## Final comments and summary 

### Important ideas to know

- RNNS and LSTMs are well suited for sequence modeling tasks. 
- They are widely used models to process sequential data in deep learning community and have a wide range of applications. 
- Training RNNs is hard because of vanishing and exploding gradients.  
- LSTMs mitigate the problem by introducing a mechanism to selectively control the flow of information in the network.   

### Important ideas to know

- LSTMs use **gates to control the flow of information**. 
- They maintain **a separate cell or context state** in addition to the hidden state. 
- Maintaining cell or context state allows for efficient training with back-propagation through time.     

### Important ideas to know

- The shapes of weight matrices ($U, V, W$) in vanilla RNNs.
- Forward pass of RNNs and LSTMs.  
- The shapes of input, output, and hidden and cell states of LSTM. 

### Important ideas to know

- If you want to use LSTMs for text data a number of steps are involved before passing text data to LSTMs including
    - tokenization
    - embedding lookup 
    - numericalization
    - batching 
    - padding 
- We'll use a library called [`torchtext`](https://pytorch.org/text/stable/index.html) to help us with some of the preprocessing tasks. 

<br><br><br><br>

### RNN resources

A lot of material is available on the web. Here are some resources that were useful for me. 


- [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [An illustration of LSTMs with nice visualizations](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)
- Geoff Hinton's [short talk](https://www.youtube.com/watch?v=93rzMHtYT_0) and [lecture](https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec7.pdf) on LSTMs
- [Yoshua Bengio's lecture](https://www.youtube.com/watch?v=AYku9C9XoB8&t=884s)
- [Ali Ghodsi's lecture on RNNs](https://www.youtube.com/results?search_query=ali+ghodsi+RNNs)
- Richard Socher's [slides](https://cs224d.stanford.edu/lectures/CS224d-Lecture8.pdf) and [lecture](https://www.youtube.com/watch?v=Keqep_PKrY8) on RNNs
- [A list of RNN resources](https://github.com/ajhalthor/awesome-rnn)
- [MIT 6.S191 (2020): Recurrent Neural Networks](https://www.youtube.com/watch?v=SEnXr6v2ifU)

<br><br><br><br>